In [26]:
pip install langchain langchain-community langchain-google-genai pandas python-dotenv 

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [27]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from pathlib import Path
from langchain_google_genai import ChatGoogleGenerativeAI 
# from langchain_google_genai import GoogleGenerativeAIEmbeddings
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["GOOGLE_API_KEY"]=os.getenv("GOOGLE_API_KEY")

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")
# embeddings = GoogleGenerativeAIEmbeddings()


In [28]:
import os
os.makedirs("data",exist_ok=True)

In [29]:

import pandas as pd
df = pd.read_csv("data/cx_data.csv")
df.head()

,customer_id,name,age,gender,location,subscription_type,join_date,last_login,total_purchases,avg_order_value,complaint_category,complaint_description,resolution_time_hours,satisfaction_score,support_channel,product_category,issue_severity,resolution_status,employee_id,department
0,CX001,Sarah Johnson,28,Female,New York,Premium,2023-01-15,2024-08-29,45,127.50,Billing,Incorrect charge on monthly subscription,24,3,Email,Software,Medium,Resolved,EMP101,Billing
1,CX002,Michael Chen,34,Male,San Francisco,Basic,2022-11-20,2024-08-28,23,89.25,Technical,App crashes during video calls,72,2,Phone,Software,High,Resolved,EMP205,Technical Support
2,CX003,Emily Rodriguez,41,Female,Los Angeles,Enterprise,2021-05-10,2024-08-30,156,245.80,Feature Request,Need integration with Slack,48,4,Chat,Software,Low,In Progress,EMP150,Product Development
3,CX004,David Kim,29,Male,Seattle,Premium,2023-03-22,2024-08-27,67,156.90,Billing,Payment method declined,12,4,Phone,Software,Medium,Resolved,EMP101,Billing
4,CX005,Jessica Williams,52,Female,Chicago,Basic,2022-08-14,2024-08-26,34,76.40,Technical,Cannot access dashboard,36,3,Email,Software,Medium,Resolved,EMP210,Technical Support


In [30]:
loader=CSVLoader(file_path="data/cx_data.csv")
docs = loader.load_and_split()

In [31]:
pip install faiss-cpu

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [32]:
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="gemini-embedding-001")
index = faiss.IndexFlatL2(len(GoogleGenerativeAIEmbeddings(model="gemini-embedding-001").embed_query(" ")))
vector_store = FAISS(
    embedding_function=GoogleGenerativeAIEmbeddings(model="gemini-embedding-001"),
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={}
)

In [33]:

vector_store.add_documents(documents=docs)

['9c138920-001f-426a-8f10-1416c5e9359c',
 'a3578976-da4c-4836-b2a9-210ae95d5563',
 '8b893723-fdd7-4cee-a901-72b90de138bc',
 'c6bc5953-e94a-43aa-afcb-7415a5f8ff31',
 '89d67372-dc57-4d22-8a4b-0bf820ba1cac',
 '54a56e75-2c29-45dd-beeb-04f046f2cccf',
 '8108529d-1f69-4653-b04e-24a940582e7a',
 'daaff8e9-0f81-4327-876b-9495fe099136',
 '6d276306-963a-44f9-86a0-eff80d86a46c',
 'b521368f-53b0-415a-8945-eea5ed66feb4',
 'e2cdc50a-4651-4fcb-a712-8db3e72dcf70',
 'fd34d790-0c93-445c-987e-d58ecfafcae6',
 'e1cb1914-46c0-4977-bc08-40a9ff8d002d',
 '5fd9027d-4a3d-4377-9abf-ec4f9c4fdc2d',
 'c7256331-77e8-47cb-897b-44b845b68d5e',
 '22f438d2-539e-40b8-b7d0-c0a774e68ae4',
 '6392e02f-25f9-4961-ab40-e2fb1668ce9a',
 '99699af8-aa73-4e96-930b-c8b4fffc127e',
 '01c08f77-28a0-4f7d-8f0d-071d63d8d3d7',
 'f53503e7-e4d5-41aa-9c9d-bce067cd80f0',
 'c56e3595-401b-4ba9-9b5b-8cf634e762d1',
 '8150e62e-3c88-48e7-8b1b-e8707844554d',
 '72e58f5e-35ac-404e-99b0-d531587bfbf7',
 'c5db1e97-e890-415b-a635-39521f37ce51',
 'd52ec1b2-999c-

In [34]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

retriever=vector_store.as_retriever()

system_prompt=(
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
    )

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, prompt)
retrieval_chain = create_retrieval_chain(retriever, question_answer_chain)

In [37]:
answer = retrieval_chain.invoke({"input": "What is the most common reason for customer complaints?"})
print(answer,"this is context")
print(answer["answer"])

{'input': 'What is the most common reason for customer complaints?', 'context': [Document(id='9c138920-001f-426a-8f10-1416c5e9359c', metadata={'source': 'data/cx_data.csv', 'row': 0}, page_content='customer_id: CX001\nname: Sarah Johnson\nage: 28\ngender: Female\nlocation: New York\nsubscription_type: Premium\njoin_date: 2023-01-15\nlast_login: 2024-08-29\ntotal_purchases: 45\navg_order_value: 127.50\ncomplaint_category: Billing\ncomplaint_description: Incorrect charge on monthly subscription\nresolution_time_hours: 24\nsatisfaction_score: 3\nsupport_channel: Email\nproduct_category: Software\nissue_severity: Medium\nresolution_status: Resolved\nemployee_id: EMP101\ndepartment: Billing'), Document(id='02ad24cb-2c70-4876-9fd7-466b7230a71a', metadata={'source': 'data/cx_data.csv', 'row': 39}, page_content='customer_id: CX040\nname: Marcus Turner\nage: 45\ngender: Male\nlocation: Virginia Beach\nsubscription_type: Enterprise\njoin_date: 2021-05-27\nlast_login: 2024-08-28\ntotal_purchases: